In [1]:
import pandas as pd
import numpy as np
import difflib

In [2]:
## Construct category list
#program_f = pd.read_csv('handson_data/handson_program.csv')
cats = ['Assistance and Support for Elderly',
        'Disaster & Emergency Services',
        'Education and Empowerment for Children and Youth',
        'Empowerment and Support for Domestic and Migrant Workers', 
        'Environmental Conservation',
        'Education and Empowerment for Refugees and Asylum Seekers',
        'Health and Wellness',
        'Hunger & Homelessness', 
        'Support for Homeless People',
        'Adult Education',
        'International Service',
        'Internships & Employment',
        'Justice & Legal Services',
        'Refugee & Asylum Seekers Services',
        'Schools',
        'Sports & Recreation',
        'Technology',
        'Family Services',
        'Arts & Culture',
        'Civic & Community']
print(len(cats))
cats_abbr = ['interest_elderly','interest_disaster','interest_education_young',
             'interest_worker','interest_environment','interest_education_refugee',
             'interest_health','interest_hunger','interest_homeless',
             'interest_education_adult','interest_international','interest_internship',
             'interest_justice','interest_refugee','interest_school','interest_sports',
             'interest_tech','interest_family','interest_arts','interest_civic']
cats_dict = {cats_abbr[i]:cats[i] for i in range(len(cats))}

20


In [3]:
cats_dict

{'interest_elderly': 'Assistance and Support for Elderly',
 'interest_disaster': 'Disaster & Emergency Services',
 'interest_education_young': 'Education and Empowerment for Children and Youth',
 'interest_worker': 'Empowerment and Support for Domestic and Migrant Workers',
 'interest_environment': 'Environmental Conservation',
 'interest_education_refugee': 'Education and Empowerment for Refugees and Asylum Seekers',
 'interest_health': 'Health and Wellness',
 'interest_hunger': 'Hunger & Homelessness',
 'interest_homeless': 'Support for Homeless People',
 'interest_education_adult': 'Adult Education',
 'interest_international': 'International Service',
 'interest_internship': 'Internships & Employment',
 'interest_justice': 'Justice & Legal Services',
 'interest_refugee': 'Refugee & Asylum Seekers Services',
 'interest_school': 'Schools',
 'interest_sports': 'Sports & Recreation',
 'interest_tech': 'Technology',
 'interest_family': 'Family Services',
 'interest_arts': 'Arts & Culture

In [4]:
## construct skill set
skills = ['Accounting / finance',
          'Administrative/general office support',
          'Communications and marketing',
          'Fundraising',
          'Gardening',
          'Graphic design',
          'Health / medical',
          'Event management',
          'Human resources',
          'IT',
          'Legal',
          'Photography',
          'Project management',
          'Teaching/Training/Fine arts',
          'Teaching/Training/Fitness/sports',
          'Teaching/Training/Music/drama',
          'Teaching/Training/Health',
          'Teaching/Training/IT',
          'Translation',
          'Videography']
print(len(skills))

20


In [41]:
## Construct feature map for user
feature = ['gender_female','gender_male',
           'employment_pt','employment_ft','employment_retired','employment_others',
           'hours_served_total','hours_served_oppo','oppo_cat',
           'interest_elderly','interest_disaster','interest_education_young',
           'interest_worker','interest_environment','interest_education_refugee',
           'interest_health','interest_hunger','interest_homeless',
           'interest_education_adult','interest_international','interest_internship',
           'interest_justice','interest_refugee','interest_school','interest_sports',
           'interest_tech','interest_family','interest_arts','interest_civic']
print(len(feature))

29


In [6]:
## Import user & program data
user_f = pd.read_csv('handson_data/dummy_volunteer_profile.csv')
program_f = pd.read_csv('handson_data/handson_program.csv')

In [7]:
## Map between event name and cat
program_cat_match = program_f[['impact_area','opportunity_name']]
program_cat_match = program_cat_match.dropna(how='all')
program_cat_match = program_cat_match.drop_duplicates()
program_cat_match

,impact_area,opportunity_name
0,Environmental Conservation,Serve-A-Thon: Soap Recycling 1:30 p.m. session...
2,Environmental Conservation,Serve-A-Thon: Soap Recycling 10 a.m. session (...
4,Hunger & Homelessness,Serve-A-Thon: Deliver Daily Necessities to Hom...
6,Environmental Conservation,Serve-A-Thon: Help with sorting donated goods ...
13,Hunger & Homelessness,Youth Opportunity: Collect Bread for Redistrib...
...,...,...
462,Hunger & Homelessness,Youth Empowered Opportunity: Collect Bread for...
468,Education and Empowerment for Children and Youth,Youth Empowered Info Session: Providing suppor...
469,Environmental Conservation,Youth Empowered Opportunity: Info Session on R...
470,Education and Empowerment for Children and Youth,Youth Empowered Info Session: Be a 'Blind Budd...


In [8]:
## Turn map to dict
program_cat_match_dict = program_cat_match.set_index('opportunity_name').T.to_dict('list')

<ipython-input-8-8fef1c2c1726>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  program_cat_match_dict = program_cat_match.set_index('opportunity_name').T.to_dict('list')


In [30]:
## Construct feature matrix from user data
user_matrix_id = user_f.connection_id
user_matrix_gender_female = user_f.gender.str.contains('Female')
user_matrix_gender_male = user_f.gender.str.contains('Male')
user_matrix_employment_pt = user_f.employment_status.str.contains('Part Time')
user_matrix_employment_ft = user_f.employment_status.str.contains('Full Time')
user_matrix_employment_retired = user_f.employment_status.str.contains('Retired')
d = {'Other': True, 'Not Employed': True, 'Full Time': False, 'Part Time': False}
user_matrix_employment_others = user_f.employment_status.map(d)
user_matrix_employment_others = user_matrix_employment_others.fillna(True)
user_matrix_hours_served_total = user_f.hours_served_all
user_matrix_hours_served_oppo = user_f.hours_served_opportunity
user_matrix_oppo_cat = user_f.opportunity_name.map(program_cat_match_dict).rename('oppo_cat')

In [31]:
## Construct feature matrix from user interest
for cat_abbr in cats_abbr:
    tmp_pdseries = user_f.interest_list.str.contains(cats_dict[cat_abbr])
    tmp_pdseries = tmp_pdseries.fillna(False)
    vars()['user_matrix_'+cat_abbr] = tmp_pdseries

In [34]:
## Build feature matrix for user
user_matrix = pd.DataFrame(user_matrix_id)
for attr in feature:
    user_matrix = pd.concat([user_matrix,vars()['user_matrix_'+attr]],axis=1)
    
user_matrix = user_matrix.replace({True: 1, False: 0})

In [38]:
user_matrix

,connection_id,gender,gender,employment_status,employment_status,employment_status,employment_status,hours_served_all,hours_served_opportunity,oppo_cat,...,interest_list,interest_list,interest_list,interest_list,interest_list,interest_list,interest_list,interest_list,interest_list,interest_list
0,CO-095401,1,0,0.0,0.0,0.0,1,25.24,1.50,NaN,...,0,0,0,0,0,0,0,0,0,0
1,CO-095078,1,0,0.0,0.0,0.0,1,25.24,1.50,NaN,...,0,0,0,0,0,0,0,0,0,0
2,CO-093654,1,0,0.0,0.0,0.0,1,25.24,1.50,NaN,...,0,0,0,0,0,0,0,0,0,0
3,CO-095194,1,0,0.0,0.0,1.0,1,13.76,2.00,NaN,...,0,1,0,0,0,0,0,0,0,0
4,CO-094334,1,0,0.0,0.0,1.0,1,13.76,1.17,NaN,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,CO-094842,1,0,0.0,0.0,0.0,1,111.40,1.25,NaN,...,1,0,0,0,1,0,0,1,1,0
95,CO-094841,1,0,0.0,0.0,0.0,1,111.40,1.25,NaN,...,1,0,0,0,1,0,0,1,1,0
96,CO-094836,1,0,0.0,0.0,0.0,1,111.40,0.00,NaN,...,1,0,0,0,1,0,0,1,1,0
97,CO-093631,1,0,1.0,0.0,0.0,0,9.50,3.00,[Hunger & Homelessness ],...,1,0,0,0,0,0,0,1,0,0


In [36]:
## Build Transform matrix
transform_mat = np.random.rand(len(cats),len(feature))

array([[0.25968229, 0.92455361, 0.76938346, 0.26120322, 0.62753331,
        0.05897312, 0.66394635, 0.9349879 , 0.69461099, 0.23594267,
        0.07207483, 0.5012181 , 0.44316127, 0.93233181, 0.4606958 ,
        0.88090778, 0.82928798, 0.55052581, 0.35188593, 0.50306667,
        0.40992502, 0.00430385, 0.48068538, 0.8053306 , 0.77078175,
        0.01638356, 0.7193655 , 0.33922523, 0.29564595],
       [0.00107204, 0.16734522, 0.09187549, 0.52224453, 0.62790763,
        0.9856143 , 0.39779045, 0.49917642, 0.61044694, 0.91551141,
        0.79522106, 0.75073363, 0.3574355 , 0.52153853, 0.91084837,
        0.54218366, 0.08469506, 0.85422901, 0.549461  , 0.40148324,
        0.45180037, 0.54874458, 0.53598842, 0.51601027, 0.33804328,
        0.00270112, 0.7487324 , 0.75626458, 0.98258194],
       [0.49281386, 0.63541092, 0.7805121 , 0.59886303, 0.39327376,
        0.65987884, 0.15693245, 0.84413459, 0.94915298, 0.04026341,
        0.7938604 , 0.9589367 , 0.15980504, 0.0044164 , 0.69567202,
  

In [ ]:
## Generate prediction result
for i in range(len(user_matrix)):
    pred_result_one = np.multiply(transform_mat,np.array(user_matrix.iloc[i].drop('connection_id')))
    highest_three_one = 